In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('twitter_training.csv', sep=',', names=["Tweet ID", "Entity", "Sentiment", "Tweet Content"])
df.head()

,Tweet ID,Entity,Sentiment,Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [3]:
#Data cleaning
df["Tweet ID"] = range(1, len(df) + 1)
df.head()

,Tweet ID,Entity,Sentiment,Tweet Content
0,1,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2,Borderlands,Positive,I am coming to the borders and I will kill you...
2,3,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,4,Borderlands,Positive,im coming on borderlands and i will murder you...
4,5,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
#To-do review the lectures for NLP/Tokenazation and Lemmatization

# Miguel - look into pipelines